
#### Run the cell below to install the required packages for Copilot


In [ ]:
%pip install openai

In [ ]:
import os
from  openai import AzureOpenAI

# Define the path you want to list directories from
path = "/lakehouse/default/Files/synthetic_data"

# List all directories in the given path
directories = [path+"/"+d for d in os.listdir(path)]# if os.path.isdir(os.path.join(path, d))]

print(directories)

# Remove the prefix '/lakehouse/default/'
directories = [path.replace('/lakehouse/default/', '') for path in directories]

In [ ]:
directories

In [43]:
# Extract sample data and column names
def get_sample_data(df, sample_size=5):
    sample_data = df.limit(sample_size)
    return sample_data

StatementMeta(, 6a85ae19-31db-4eed-bd85-c58ae4132d10, 60, Finished, Available, Finished)

In [54]:
fake_functions = ['first_name', 'last_name', 'email', 'phone_number', 'ssn', 'passport_number', 'street_address', 'name', 'user_name', 'date_of_birth'] #from Faker librarary
expected_output=  """"pii_column_mapping": {
            "CustomerID": "random_int",
            "FirstName": "first_name",
            "LastName": "last_name"
        }"""

def create_big_prompt(df,sample_size=5):
    # Start the prompt with an introduction
    big_prompt = """Based on the following column names, their sample data and fake function names from faker library, identify the columns 
                    containing PII (Personally Identifiable Information) and return result in given format,
                    without any code formatting or extra text. If No column contains PII return None as response.\n\n"""
    for column in df.columns:
        # Collect sample data for the column
        sample_data = [row[column] for row in df.select(column).distinct().limit(sample_size).collect()]
        
        # Add the column and its sample data to the prompt
        big_prompt += f"Column Name: {column}\n"
        big_prompt += f"Sample Data: {sample_data}\n\n"
        big_prompt += f"fake functions: {fake_functions}\n\n"
        
    big_prompt += f"expected_output: {expected_output}\n\n"
        

    return big_prompt


StatementMeta(, 6a85ae19-31db-4eed-bd85-c58ae4132d10, 71, Finished, Available, Finished)

In [55]:
def classify_columns_with_llm(classify_columns_with_llm):
    endpoint = "https://<your open AI>.openai.azure.com/" 
    deployment = "gpt-4o"  
    subscription_key = ""

    # Initialize Azure OpenAI client with key-based authentication    
    client = AzureOpenAI(  
        azure_endpoint=endpoint,  
        api_key=subscription_key,  
        api_version="2024-05-01-preview",  
    )
        
        
    #Prepare the chat prompt 
    chat_prompt = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": pii_columns_in_df_prompt
                }
            ]
        }
    ] 
        
    # Include speech result if speech is enabled  
    messages = chat_prompt  
        
    # Generate the completion  
    completion = client.chat.completions.create(  
        model=deployment,  
        messages=messages,  
        max_tokens=800,  
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,  
        stop=None,  
        stream=False
    )

    #print(completion.to_json())
    return completion.choices[0].message.content

StatementMeta(, 6a85ae19-31db-4eed-bd85-c58ae4132d10, 72, Finished, Available, Finished)

In [57]:

from pyspark.sql.utils import AnalysisException

# Iterate over the list of paths and check for PII columns
for path in directories:
    try:
        # Read the DataFrame from the given path (assuming Delta format)
        original_df = spark.read.load(path)

        sample_df = get_sample_data(original_df)

        pii_columns_in_df_prompt = create_big_prompt(sample_df)

        pii_columns_in_df = classify_columns_with_llm(pii_columns_in_df_prompt)
        display(sample_df)
        print(pii_columns_in_df)

        #anomnised_df = anomnise_data(original_df,pii_columns_in_df)

        #save_anomised_data(anomnised_df)

    except AnalysisException:
        print(f"Could not read DataFrame from {path}. Ensure the path exists or the format is correct.")


StatementMeta(, 6a85ae19-31db-4eed-bd85-c58ae4132d10, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 77ee44a4-8edc-4fed-8454-79a36e530bcc)

"pii_column_mapping": {
    "FirstName": "first_name",
    "LastName": "last_name",
    "Email": "email",
    "PhoneNumber": "phone_number",
    "DateOfBirth": "date_of_birth"
}


SynapseWidget(Synapse.DataFrame, cc44f4b5-88cc-48f3-b3e6-cd53c5137903)

None


...

None


SynapseWidget(Synapse.DataFrame, a8d21162-9729-43db-9028-fcb73eae9a46)

"pii_column_mapping": {
    "ManagerName": "name"
}


SynapseWidget(Synapse.DataFrame, 62cb11a2-a53b-4f45-a82e-cb27c0f83fa7)

{
    "pii_column_mapping": {
        "CustomerEmail": "email"
    }
}
